# 再訓練されたYOLOモデルのテスト

モデルを再訓練したので、'テスト'画像に対してテストできます。

In [ ]:
# このラボ用に設計されたWorkbenchイメージを使用していない場合は、以下の行のコメントを外して実行することで、必要なパッケージをインストールできます。
# !pip install --no-cache-dir --no-dependencies -r requirements.txt

from ultralytics import YOLO
from PIL import Image
import numpy as np
import cv2
from matplotlib import pyplot as plt

import remote_infer

前のノートブックでは、モデルを再訓練する方法を見ました。これは効率的に行うためにGPUを必要とする時間のかかる作業ですので、ワークショップ中には実行できません。

そのため、既に再訓練されたモデルがONNX形式で利用可能です。

In [ ]:
# 新しく訓練されたモデルを取得します。

model = YOLO("https://rhods-public.s3.amazonaws.com/demo-models/ic-models/accident/accident_detect.onnx", task="detect")

In [ ]:
# 車の事故画像に対してモデルをテストします
image_path = "images/carImage3.jpg" # Image3は86%以上の確率で重度の事故を表していることがわかっています。

_, scale, original_image = remote_infer.preprocess(image_path)

original_image: np.ndarray = cv2.imread(image_path)
blob = cv2.dnn.blobFromImage(original_image, size=(640, 640), swapRB=False)
blob = np.ascontiguousarray(blob[0].transpose((1,2,0)))
results = model.predict(blob)

In [ ]:
# 結果からすべての情報（タイプ、バウンディングボックス、確率）を抽出します

detections = []
result = results[0]
for box in result.boxes:
    class_id = int(box.cls.item())
    score = box.conf.item()
    unscaled_cords = box.xyxy.squeeze().tolist()
    cords = [round(unscaled_cords[0] * scale[1]), round(unscaled_cords[1] * scale[0]), round(unscaled_cords[2] * scale[1]), round(unscaled_cords[3] * scale[0])]
    detection = {
        'class_id': class_id,
        'class_name': result.names[class_id],
        'confidence': score,
        'box': cords,
        'scale': scale}
    detections.append(detection)
    print(detection)
    remote_infer.draw_bounding_box(original_image, class_id, score, cords[0], cords[1], cords[2], cords[3])

In [ ]:
# 画像にボックス、クラスの名前、および確率（モデルが検出にどれだけ自信を持っているか）を描画します。

img = cv2.cvtColor(original_image, cv2.COLOR_BGR2RGB)
fig = plt.gcf()
fig.set_size_inches(6, 3)
plt.axis('off')
plt.imshow(img)

'carImage3.jpg'を調べると、再訓練されたYOLOモデルが86%の確信を持って'車の事故'を正しく予測します。次に、事故の周りにボックスが描画され、`severe 0.86`とラベル付けされます。

事故の重症度を検出できるモデルができたので、予測関数を作成し、ModelMeshでモデルを提供しましょう。

**ラボの指示に戻って、それを行う方法を学んでください。**

**ノートブック`04-05-model-serving.ipynb`はまだ開かないでください**。